In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import yaml
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import precision_recall_curve, roc_curve, average_precision_score, roc_auc_score

%matplotlib inline

In [ ]:
tpm_fmt = '../seqc/output/seqc-bgi/gene_quants/{sample}/vbprior={vb_prior}/{fold}/gene_quants.sf'
SAMPLES = [
    'BGI_FC1_A_1',
    'BGI_FC1_A_2',
    'BGI_FC1_A_3',
    'BGI_FC1_A_4',
    'BGI_FC1_B_1',
    'BGI_FC1_B_2',
    'BGI_FC1_B_3',
    'BGI_FC1_B_4',
    'BGI_FC1_C_1',
    'BGI_FC1_C_2',
    'BGI_FC1_C_3',
    'BGI_FC1_C_4',
    'BGI_FC1_D_1',
    'BGI_FC1_D_2',
    'BGI_FC1_D_3',
    'BGI_FC1_D_4' ]
VB_PRIORS = '1e-6 1e-5 1e-4 1e-3 1e-2 1e-1 1e0 2 3 4 5 6 7 8 9 1e1 1e2'.split()


In [ ]:
taqman_fp ='../seqc/data/seqc_taqman_data.tsv'
taqman_df = pd.read_csv(taqman_fp, sep='\t')
taqman_df.head()

In [ ]:
len(set(taqman_df['ensembl_gene_id']))

In [ ]:
len(set(taqman_df['EntrezID']))

In [ ]:
del taqman_df['EntrezID']
del taqman_df['Symbol']

In [ ]:
taqman_df = taqman_df.groupby('ensembl_gene_id').mean().reset_index()

In [ ]:
taqman_df = taqman_df.sort_values(by='ensembl_gene_id').set_index('ensembl_gene_id')

In [ ]:
taqman_df

In [ ]:
taqman_genes = set(taqman_df.index)

quant_fp = '../seqc/output/seqc-bgi/gene_quants/BGI_FC1_{cond}_{rep}/vbprior={vb_prior}/{fold}/gene_quant.sf'
def strip_ensembl_ver(name):
    return name.split('.')[0]

def quant_data(vb_prior, fold):
    all_df = pd.DataFrame()
    for cond in ['A', 'B', 'C', 'D']:
        for rep in range(1,5):
            df = pd.read_csv(quant_fp.format(cond=cond, rep=rep, vb_prior=vb_prior, fold=fold), sep='\t',
                            index_col=None, usecols=['abundance']).reset_index()
            df['index'] = df['index'].apply(strip_ensembl_ver)
            df = df[df['index'].isin(taqman_genes)]
            all_df['index'] = df['index']
            all_df['{}{}'.format(cond, rep)] =  df['abundance']
    all_df = all_df.sort_values(by='index').set_index('index')
    return all_df
quant_data('1e0', 1)

In [ ]:
def arr_spearman(A, B):
    return spearmanr(A, B).correlation

def spearman_corrs(df1, df2):
    assert(df1.values.shape == df2.values.shape)
    
    spearmans = [arr_spearman(df1.values[:, i], df2.values[:, i]) for i in range(df1.shape[1])]
    return spearmans

def calc_spearman_corrs(taqman_df, VB_PRIORS=VB_PRIORS):
    df = pd.DataFrame()
    for vb_prior in VB_PRIORS:
        for fold in range(1, 6):
            quants = quant_data(vb_prior, fold)
            
            spearmans = spearman_corrs(taqman_df, quants)
            N = len(spearmans)
            df = df.append(pd.DataFrame(dict(Spearman_r=spearmans, 
                           vb_prior=[vb_prior] * N,
                           fold=[fold] *N,
                           cond=['A']*4 + ['B'] * 4 + ['C'] * 4 + ['D'] * 4)), ignore_index=True)
    return df
            
    
spearman_df = calc_spearman_corrs(taqman_df, VB_PRIORS)

In [ ]:
spearman_df.vb_prior = spearman_df.vb_prior.astype(float)

In [ ]:
# ax = sns.violinplot(x="vb_prior", y="Spearman_r", data=spearman_df, inner=None, hue="cond")
# ax = sns.swarmplot(x="vb_prior", y="Spearman_r", data=spearman_df,
#                    color="white", edgecolor="gray", dodge=True, s=1, hue="cond")
# ax.set_title('SEQC Spearman Corr. vs. Microarray')

In [ ]:
mean_fold_df = spearman_df.groupby(['cond', 'vb_prior', 'fold']).mean().reset_index()

In [ ]:
mean_fold_df

In [ ]:
for cond in 'A B C D'.split():
    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
#     ax.ticklabel_format(axis='x', style='sci', sci)
    ax = sns.violinplot(x="vb_prior", y="Spearman_r", data=mean_fold_df[mean_fold_df.cond == cond], inner=None, ax=ax)
    ax = sns.swarmplot(x="vb_prior", y="Spearman_r", data=mean_fold_df[mean_fold_df.cond == cond],
                       color="white", edgecolor="gray", dodge=True, s=3, ax=ax)
    ax.set_title('Spearman Corr. of RNASeq estimates vs. Microarray ({})'.format(cond), fontsize=20)
        
    ax.set_ylabel('Spearman Correlation', fontsize=20)
    ax.set_xlabel('VBEM prior size', fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_xticklabels('1e-6 1e-5 1e-4 1e-3 1e-2 1e-1 1 2 3 4 5 6 7 8 9 1e1 1e2'.split())
    plt.show()
    

In [ ]:
mean_df = spearman_df.groupby(['cond', 'vb_prior']).mean().reset_index()

In [ ]:
for cond in 'A B C D'.split():
    fig, ax = plt.subplots(1, 1, figsize=(10, 7))
    ax = sns.boxplot(x="vb_prior", y="Spearman_r", data=mean_fold_df[mean_fold_df.cond == cond])
    ax = sns.swarmplot(x="vb_prior", y="Spearman_r", data=mean_fold_df[mean_fold_df.cond == cond],
                       color="white", edgecolor="gray", dodge=True, s=1)
    ax.set_title('SEQC Spearman Corr. vs. Microarray ({})'.format(cond))
    ax.set_ylabel('Spearman Correlation', fontsize=20)
    ax.set_xlabel('VBEM prior size', fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=12)
    ax.set_xticklabels('1e-6 1e-5 1e-4 1e-3 1e-2 1e-1 1 2 3 4 5 6 7 8 9 1e1 1e2'.split())
    plt.show()